# What is a neural network
### Neural network is a collection of neurons that are connected by layers. 
### Each neuron is a small computing unit that performs simple calculations to collectively solve a problem.
### They are organized in layers. There are 3 types of layers: input layer, hidden layer and outter layer.
### Each layer contains a number of neurons, except for the input layer. 
### Neural networks mimic the way a human brain processes information.



# Components of a neural network
### Activation function determines whether a neuron should be activated or not. 
### The computations that happen in a neural network include applying an activation function.
### If a neuron activates, then it means the input is important. The are different kinds of activation 
### functions. The choice of which activation function to use depends on what you want the output to be. 
### Another important role of an activation function is to add non-linearity to the model.
### Binary used to set an output node to 1 if function result is positive and 0 
### if the function result is negative.  

 
## Sigmod is used to predict the probability of an output node being between 0 and 1.  
 
## Tanh is used to predict if an output node is between 1 and -1. Used in classification use cases.  

## ReLU used to set the output node to 0 if fuction result is negative and keeps the result value if the result is a positive value.  
 
### Weights influence how well the output of our network will come close to the expected output value. 
### As an input enters the neuron, it gets multiplied by a weight value and the resulting output is either
### observed, or passed to the next layer in the neural network. Weights for all neurons in a layer are 
### organized into one tensor

### Bias makes up the difference between the activation function's output and its intended output. 
### A low bias suggest that the network is making more assumptions about the form of the output,
### whereas a high bias value makes less assumptions about the form of the output.

# Build a neural network
### Neural networks are comprised of layers/modules that perform operations on data. 
### The torch.nn namespace provides all the building blocks you need to build your own neural network. 
### Every module in PyTorch subclasses the nn.Module. A neural network is a module itself that consists of 
### other modules (layers). This nested structure allows for building and managing complex architectures easily.



In [1]:
%matplotlib inline
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
# Check cuda env
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cpu device


# Define the class
### We define our neural network by subclassing nn.Module, and initialize the neural network layers in __init__. Every nn.Module subclass implements the operations on input data in the forward method.

## Our neural network are composed of the following:

### The input layer with 28x28 or 784 features/pixels.
### The first linear module takes the input 784 features and transforms it to a hidden layer with 512 features
### The ReLU activation function will be applied in the transformation
### The second linear module take 512 features as input from the first hidden layer and transforms it to the next hidden layer with 512 features
### The ReLU activation function will be applied in the transformation
### The third linear module take 512 features as input from the second hidden layer and transforms it to the output layer with 10, which is the number of classes
### The ReLU activation function will be applied in the transformation

In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

## We create an instance of NeuralNetwork, and move it to the device, and print it's structure.

In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


### To use the model, we pass it the input data. This executes the model's forward, along with some background
### operations. However, do not call model.forward() directly! Calling the model on the input returns 
### a 10-dimensional tensor with raw predicted values for each class.

### We get the prediction densities by passing it through an instance of the nn.Softmax.

### The vector of raw (non-normalized) predictions that a classification model generates,
### which is ordinarily then passed to a normalization function. 
### If the model is solving a multi-class classification problem, logits typically become an input to 
### the softmax function.

In [5]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X) 
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([9])


# Weight and Bias
### The nn.Linear module randomly initializes the weights and bias for each layer and 
### internally stores the values in Tensors.

In [6]:
print(f"First Linear weights: {model.linear_relu_stack[0].weight} \n")

print(f"First Linear weights: {model.linear_relu_stack[0].bias} \n")

First Linear weights: Parameter containing:
tensor([[-0.0048, -0.0098, -0.0285,  ...,  0.0217, -0.0355, -0.0226],
        [ 0.0176,  0.0285,  0.0145,  ...,  0.0054, -0.0354,  0.0071],
        [-0.0035,  0.0300,  0.0099,  ...,  0.0249, -0.0332, -0.0244],
        ...,
        [-0.0303,  0.0086, -0.0238,  ...,  0.0023, -0.0186, -0.0066],
        [-0.0204, -0.0249, -0.0290,  ...,  0.0281, -0.0109, -0.0083],
        [ 0.0041,  0.0069,  0.0063,  ..., -0.0252, -0.0251,  0.0172]],
       requires_grad=True) 

First Linear weights: Parameter containing:
tensor([ 1.9979e-02, -1.3111e-02,  3.3245e-02, -2.6027e-02, -3.2791e-02,
         2.8108e-02, -1.2076e-02,  1.8051e-02,  2.1053e-02,  2.5713e-02,
        -2.0482e-03,  9.8931e-03, -1.3956e-02, -1.4286e-02, -6.9253e-03,
         1.5898e-02, -3.0918e-02, -1.9073e-02,  2.1945e-02, -2.9468e-02,
         2.5033e-02, -3.0237e-02, -2.3187e-02,  2.8115e-02,  3.1781e-03,
        -1.8615e-03,  3.0900e-02, -3.2999e-02,  3.5465e-02,  3.3410e-02,
        -4.

## Model layers
### Let's break down the layers in the FashionMNIST model. To illustrate it, we will take a sample minibatch 
### of 3 images of size 28x28 and see what happens to it as we pass it through the network.

In [7]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


## nn.Flatten
### We initialize the nn.Flatten layer to convert each 2D 28x28 image into a contiguous array of 784 pixel
### values (the minibatch dimension (at dim=0) is maintained). Each of the pixels are pass to the input layer
### of the neural network.

In [8]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


## nn.Linear
### The linear layer is a module that applies a linear transformation on the input using it's stored weights
### and biases. The gayscale value of each pixel in the input layer will be connected to neurons in the
### hidden layer for calculation. The calculation used for the transformation is weights*input+bias


In [9]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


## nn.ReLU
### Non-linear activations are what create the complex mappings between the model's inputs and outputs. 
### They are applied after linear transformations to introduce nonlinearity, helping neural networks learn
### a wide variety of phenomena. In this model, we use nn.ReLU between our linear layers, but there's other 
### activations to introduce non-linearity in your model.

### The ReLU activation function takes the output from the linear layer calculation and replaces the negative 
### values with zeros.

In [10]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.3435,  0.5126,  0.4097,  0.2583, -0.0781, -0.1835,  0.3323, -0.1861,
         -0.1973,  0.2981,  0.1101,  0.1737, -0.4396,  0.0357,  0.0319, -0.3189,
         -0.0996, -0.0631,  0.0605,  0.2851],
        [ 0.0854,  0.4381,  0.1920,  0.2365,  0.1104, -0.2648, -0.0686,  0.0771,
          0.3875,  0.2974,  0.1432,  0.5018, -0.2782,  0.2520, -0.1905, -0.7304,
         -0.1839,  0.1481, -0.0445,  0.2097],
        [-0.0128,  0.5809,  0.2120,  0.3726, -0.0301, -0.3172,  0.2074, -0.2738,
          0.3363,  0.2090,  0.0271,  0.3709, -0.4761, -0.0735, -0.2454, -0.1175,
         -0.2369,  0.1966, -0.0215,  0.0459]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.3435, 0.5126, 0.4097, 0.2583, 0.0000, 0.0000, 0.3323, 0.0000, 0.0000,
         0.2981, 0.1101, 0.1737, 0.0000, 0.0357, 0.0319, 0.0000, 0.0000, 0.0000,
         0.0605, 0.2851],
        [0.0854, 0.4381, 0.1920, 0.2365, 0.1104, 0.0000, 0.0000, 0.0771, 0.3875,
         0.2974, 0.1432, 0.5018, 0.0000, 0.2520, 0.00

## nn.Sequential
### nn.Sequential is an ordered container of modules. The data is passed through all the modules in the 
### same order as defined. You can use sequential containers to put together a quick network like seq_modules.

In [11]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

## nn.Softmax
### The last linear layer of the neural network returns logits - raw values in [-infty, infty], 
### which are passed to the nn.Softmax module. The Softmax activation function is used to calculate the 
### probability of the output from the neural network. It is only used on the output layer of a neural network. The results are scaled to values [0, 1] representing the model's predicted densities for each class. dim parameter indicates the dimension along which the result values must sum to 1. The node with the highest probability predicts the desired output.

In [12]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

## Model parameters
### Many layers inside a neural network are parameterized, i.e. have associated weights and biases that are 
### optimized during training. Subclassing nn.Module automatically tracks all fields defined inside your 
### model object, and makes all parameters accessible using your model's parameters() or named_parameters()methods.

### In this example, we iterate over each parameter, and print its size and a preview of its values.

In [13]:
print("Model structure: ", model, "\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure:  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
) 


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0048, -0.0098, -0.0285,  ...,  0.0217, -0.0355, -0.0226],
        [ 0.0176,  0.0285,  0.0145,  ...,  0.0054, -0.0354,  0.0071]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([ 0.0200, -0.0131], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0380,  0.0343, -0.0201,  ..., -0.0433, -0.0052,  0.0366],
        [ 0.0227,  0.0003, -0.0110,  ..., -0.0415, -0.0271, -0.0150]],
       grad_fn=<SliceBackward0>) 

Layer: linear_re